In [4]:
import numpy as np
import xlrd
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
data_wb = xlrd.open_workbook('CHE223-Assignment1 for 2023-data sets up to 2019.xls')

data = pd.DataFrame({'Year': [2018], 'Ion': ['Ca'], 'Conc (mg/L)': [0], 'Relative Standard Deviation (RSD)': [0], 'Postal Code': [None]})